# Project - Seminar Computer Vision by Deep Learning (CS4245) 2020/2021

Group Number: 20

Student 1: Stan Zwinkels

Student 2: Ted de Vries Lentsch

Date: June 14, 2021

## Instruction

For correct functioning of this notebook, the dataset [morado_5may](https://www.kaggle.com/teddevrieslentsch/morado-5may) must be in the same directory as this notebook.

## Import necessary libraries

In [ ]:
# standard libraries
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import time

# widgets
from IPython.display import display, clear_output
import ipywidgets

## Relabel

Make folder with for the annotations with the new labels.

In [ ]:
root_path = 'morado_5may'
relabel_path = '{}/annotations_relabel'.format(root_path)
if os.path.isdir(relabel_path):
    shutil.rmtree(relabel_path)
    time.sleep(0.1)
    os.makedirs(relabel_path)
else:
    os.makedirs(relabel_path)

Below is the `ReLabelDataset` class for relabeling.

In [ ]:
class ReLabelDataset(object):
    def __init__(self, root):
        self.root       = root                                                          # directory to dataset
        self.imgs       = list(sorted(os.listdir('{}/images'.format(root))))            # load images
        self.annots     = list(sorted(os.listdir('{}/annotations'.format(root))))       # load annotations
        self.classes    = ['background', 'raw', 'ripe']                                 # classes
        self.idx        = 0                                                             # image/annotation index
        self.idx_last   = -1                                                            # last image/annotation index  
        self.row_number = -1                                                            # number of the current row
        self.start      = True                                                          # initialize process
        self.img        = None                                                          # image
        self.annot      = None                                                          # annotation
        self.done       = False                                                         # whether all images have been labeled 

    def plot_patch(self):
        with out:
            annot = self.annot.loc[self.row_number,0:4].to_numpy()
            img   = self.img[int(annot[1]):(int(annot[3])+1),int(annot[0]):(int(annot[2])+1),:]   
            
            clear_output(True)
            if not self.done:
                plt.figure(figsize=(5, 5))
                plt.imshow(img, zorder=-10)
                plt.title('Old label: {}'.format(self.annot.loc[self.row_number, 4]))
                plt.show()
            else:
                plt.figure()
                plt.show()

    def manage_ids(self):      
        if self.row_number==len(self.annot)-1:
            self.save_annot()
            self.row_number = 0
            self.idx_last = self.idx
            self.idx += 1
            if self.idx==len(self.imgs):
                self.done = True
        else:
            self.idx_last = self.idx
            self.row_number += 1

    def get_data(self):
        if self.idx!=self.idx_last:
            img_path      = '{}/images/{}'.format(self.root, self.imgs[self.idx])                   
            annot_path    = '{}/annotations/{}'.format(self.root, self.annots[self.idx])              
            self.img      = np.rot90(plt.imread(img_path), -1)                             
            self.annot    = pd.read_csv(annot_path, sep=',', header=None)   

    def save_annot(self):
        annot_re_path = '{}/annotations_relabel/{}'.format(self.root, self.annots[self.idx])
        self.annot.sort_values(by=[4], inplace=True)
        self.annot.reset_index(drop=True, inplace=True)
        self.annot.to_csv(annot_re_path, index=0, header=0)
        print('The file {} has been relabeled!'.format(self.annots[self.idx]))

    def button_click_action(self, label):
        if not self.done:
            self.get_data()
            if not self.start:
                self.annot.at[self.row_number,4] = label
            self.start = False
            self.manage_ids()
            self.plot_patch()        
        
    def left_button_click(self, click):
        self.button_click_action('raw')

    def right_button_click(self, click):
        self.button_click_action('ripe')

Below is the tool for relabeling. The process is started by clicking on one of the two buttons. The first annotation is then plotted. You can then indicate for each image to which class it belongs. If all the annotations for one image have been made, a new .csv file is saved in the `annotations_relabel` directory that was created above.

In [ ]:
%matplotlib inline

In [ ]:
relabeler = ReLabelDataset(root_path)

# create buttons for the 2 classes
button_left = ipywidgets.Button(description='Raw')
button_right = ipywidgets.Button(description='Ripe')

# assign functions to the press of the buttons
button_left.on_click(relabeler.left_button_click)
button_right.on_click(relabeler.right_button_click)

# output window for the plot
out = ipywidgets.Output()

# widget
ipywidgets.VBox([ipywidgets.HBox([button_left, button_right]), out])